<a href="https://colab.research.google.com/github/20KMJ/ESAA7/blob/main/0519_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH9 추천 시스템

# 08 | 파이썬 추천 시스템 패키지 - Surprise

## 8.1 Surprise 패키지 소개

- 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지
- Surprise 패키지 주요 장점:
1. 다양한 추천 알고리즘(사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링)을 쉽게 적용해 추천 시스템 구축 가능
2. 사이킷런의 핵심 API와 유사한 API명으로 작성됨 (fit, predict, train_test_split, cross_validate, GridSearchCV 클래스를 통한 모델 셀렉션/평가/하이퍼 파라미터 튜닝 등)

In [1]:
! pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095462 sha256=a934ef82b7825d0c8ce3dc02607b1415317d3fed32454beaea56efb2886ada56
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


## 8.2 Surprise를 이용한 추천 시스템 구축

In [4]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
data = Dataset.load_builtin('ml-100k') # 10만 개 평점 데이터
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [6]:
# SVD 잠재 요인 협업 필터링
algo = SVD()
algo.fit(trainset)

In [7]:
## test()
predictions = algo.test(testset)
print('prediction type :', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5174109320028846, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.818951355297434, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.367399735739253, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.400038290489974, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.2337845696882663, details={'was_impossible': False})]

In [13]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5174109320028846),
 ('882', '291', 3.818951355297434),
 ('535', '507', 4.367399735739253)]

In [14]:
## predict()
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.36   {'was_impossible': False}


In [15]:
accuracy.rmse(predictions)

RMSE: 0.9501


0.9500673213280812

## 8.3 Surprise 주요 모듈 소개

### 8.3.1 Dataset

1. `Dataset.load_builtin(name='ml-100k')`
  - 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터 내려받기
2. `Dataset.load_from_file(file_path, reader)`
  - OS 파일에서 데이터 로딩할 때 사용
3. `Dataset.load_from_df(df, reader)`
  - 판다스의 DataFrame에서 데이터 로딩

### 8.3.2 OS 파일 데이터를 Surprise 데이터 세트로 로딩

`Dataset.load_from_file`

- 주의: 칼럼 Header 제거 필요

In [16]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [18]:
from surprise import Reader
# 파싱 정보 제공
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

> **Reader 클래스 주요 생성 파라미터**  
> `line_format (string)` : 칼럼을 순서대로 나열함. 입력된 문자열을 공백으로 분리해 칼럼으로 인식한다.
>
>
> `sep (char)` : 칼럼을 분리하는 분리자  
>
>
> `rating_scale (tuple, optional)` : 평점 값의 최소~최대 평점 설정

In [19]:
# SVD 행렬 분해 기법을 이용한 추천 예측

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0) # n_factors는 잠재 요인 크기 K값을 나타냄
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 8.3.3 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [20]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## 8.4 Surprise 추천 알고리즘 클래스

Surprise에서 추천 예측을 위해 자주 사용되는 추천 알고리즘 클래스:
1. **SVD**: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
2. **KNNBasic**: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
3. **BaselineOnly**: 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

> **SVD 클래스 입력 파라미터**  
> `n_factors` : 잠재 요인 K의 개수. 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있다.
>
>
> `n_epochs` : SGD(Stochastic Gradient Descent) 수행 시 반복 횟수. 디폴트는 20.
>
>
> `biased (bool)` : 베이스라인 사용자 편향 적용 여부. 디폴트는 True

추천 알고리즘의 예측 성능 벤치마크 결과:
- SVD++ 알고리즘의 RMSE, MAE 성적이 가장 좋지만, 상대적으로 시간이 너무 오래 걸려 데이터가 조금만 더 커져도 사용하기 어려울 것으로 보임.
- SVD++를 제외하면 SVD와 k-NN Baseline이 가장 성능 평가 수치가 좋음 (k-NN 자체는 성능이 상대적으로 뒤지지만, Baseline을 결합한 경우 성능 평가 수치가 대폭 향상됨)
- *Baseline은 각 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식을 말함*

## 8.5 베이스라인 평점 (Baseline Rating)

- 개인이 평점을 부여하는 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것
- **베이스라인 평점** = 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수
1. 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
2. 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
3. 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

## 8.6 교차 검증과 하이퍼 파라미터 튜닝

In [21]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8706  0.8798  0.8688  0.8783  0.8714  0.8738  0.0044  
MAE (testset)     0.6692  0.6765  0.6647  0.6762  0.6683  0.6710  0.0046  
Fit time          2.38    1.76    1.57    1.71    1.58    1.80    0.30    
Test time         0.44    0.14    0.29    0.13    0.13    0.23    0.12    


{'test_rmse': array([0.87062063, 0.87978585, 0.86879056, 0.87829694, 0.87141276]),
 'test_mae': array([0.66923692, 0.67647479, 0.66469423, 0.67615606, 0.66830813]),
 'fit_time': (2.377865791320801,
  1.7557728290557861,
  1.5685608386993408,
  1.7095577716827393,
  1.5844125747680664),
 'test_time': (0.44184255599975586,
  0.14372467994689941,
  0.29402709007263184,
  0.13045215606689453,
  0.12713170051574707)}

In [22]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8788131854699582
{'n_epochs': 20, 'n_factors': 50}


## 8.7 Surprise를 이용한 개인화 영화 추천 시스템 구축

> Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현해보자.
- Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은(관람하지 않은) 영화 중에서 개인 취향에 가장 적절한 영화 추천하기

In [23]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

In [25]:
# 데이터 세트 전체를 학습 데이터로 사용 -> DatasetAutoFolds
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

In [27]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [28]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [29]:
# movieId 42인 영화에 대해서 userId 9 사용자의 추천 예상 평점 구하기
uid = str(9)
iid = str(42)

pred = algo.predict(uid,iid,verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


> 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화를 추천해보자.

In [30]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성.
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
        '전체 영화 수:', len(total_movies))
  
  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [31]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # prediction list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
  # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id, title, rating) for id, title, rating in
                     zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    

  return top_movie_preds

In [33]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
